In [1]:
import ee
import geemap

In [2]:
Map_S5P = geemap.Map()
Map_S5P

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
point_S5P = ee.Geometry.Rectangle([-122.8003, 37.4831, -122.8036, 37.8288])

image_S5P = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_NO2") \
    .filterBounds(point_S5P) \
    .filterDate('2018-07-11', '2019-06-30') \
    .sort('ALGORITHM_VERSION') \
    .first() \
    .select('tropospheric_NO2_column_number_density')
    #.select('NO2_column_number_density','tropospheric_NO2_column_number_density','stratospheric_NO2_column_number_density')

vis_params_S5P = {
    'min': 0,
    'max': 0.0002,
    'bands': ['tropospheric_NO2_column_number_density'],
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}
Map_S5P.centerObject(point_S5P, 8)
Map_S5P.addLayer(image_S5P, vis_params_S5P, "Sentinel-5P TROPOMI NO2")


In [4]:
Map_LANDSAT8 = geemap.Map()
Map_LANDSAT8

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [5]:
point_LANDSAT8 = ee.Geometry.Point([-122.4439, 37.7538])

image_LANDSAT8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(point_LANDSAT8) \
    .filterDate('2018-07-01', '2019-06-30') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

vis_params_L8 = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map_S5P.centerObject(point_LANDSAT8, 8)
Map_S5P.addLayer(image_LANDSAT8, vis_params_L8, "Landsat-8")


In [6]:
ground_truth= image_LANDSAT8.addBands(image_S5P)

In [7]:
#tropimi = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image_S5P.geometry())
Map_LANDSAT8.addLayer(image_S5P, {}, 'TROPOMI')
Map_LANDSAT8

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [8]:
# Make the training dataset.
points = ground_truth.sample(**{
    'region': ground_truth.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map_S5P.addLayer(points, {}, 'training', False)

In [9]:
print(points.size().getInfo())

3327


In [10]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-121.68730640159825, 37.86722226931812]}, 'id': '0', 'properties': {'B1': 652, 'B2': 889, 'B3': 1427, 'B4': 2152, 'B5': 3359, 'B6': 3883, 'B7': 2410, 'tropospheric_NO2_column_number_density': 6.197282345965505e-05}}


In [11]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'tropospheric_NO2_column_number_density'

# Overlay the points on the imagery to get training.
training = ground_truth.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileRandomForest(10).setOutputMode('REGRESSION').train(training, label, bands)

In [12]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 652, 'B2': 889, 'B3': 1427, 'B4': 2152, 'B5': 3359, 'B6': 3883, 'B7': 2410, 'tropospheric_NO2_column_number_density': 6.197282345965505e-05}}


In [13]:
vis_params_result = {
    'min': 0,
    'max': 0.0002,
    'bands': ['classification'],
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']}

In [14]:
# Classify the image with the same bands used for training.
result = ground_truth.select(bands).classify(trained)

# # Display the clusters with random colors.
Map_LANDSAT8.addLayer(result.randomVisualizer(),vis_params_result, 'classfied')
Map_LANDSAT8

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [15]:
print(result.getInfo())

{'type': 'Image', 'bands': [{'id': 'classification', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7671, 7811], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 462885, 0, -30, 4264515]}], 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[-121.05673624299249, 37.42117167346286], [-121.0419495518993, 37.46824264602141], [-120.9915192018566, 37.62853330576134], [-120.95614963963575, 37.740664939858924], [-120.89304639350827, 37.93996325648889], [-120.83570953511274, 38.120623813239646], [-120.83234044125189, 38.13156056012753], [-120.83229162963826, 38.133209000349275], [-122.58732094879598, 38.46611548597422], [-122.90566430509564, 38.5233791935817], [-122.90782331761598, 38.51662360524393], [-122.91058851970752, 38.50743070907603], [-123.39555735783479, 36.80459111753288], [-123.39554957012268, 36.80307888243811], [-122.00237455054017, 36.5398005962312], [-121.40924608814669, 36.42221797177954], [-121.36923060278302, 36.41418371264349], 

In [16]:
vis_params_diff = {
    'min': 0,
    'max': 0.0002,
    'bands': ['tropospheric_NO2_column_number_density'],
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']}

In [17]:
#// Load a 5-year Landsat 7 composite 2008-2012.
#landsat2008 = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012');

#// Compute multi-band difference between the 2008-2012 composite and the
#// previously loaded 1999-2003 composite.
diff = image_S5P.subtract(result);
#Map.addLayer(diff,
            # {bands: ['', 'B3', 'B2'], min: -32, max: 32}, 'difference');

#// Compute the squared difference in each band.
squaredDifference = diff.pow(2);
#Map.addLayer(squaredDifference,
             #{bands: ['B4', 'B3', 'B2'], max: 1000}, 'squared diff.');

In [18]:
print(diff.getInfo())

{'type': 'Image', 'bands': [{'id': 'tropospheric_NO2_column_number_density', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [265, 214], 'origin': [1115, 1073], 'crs': 'EPSG:4326', 'crs_transform': [0.01, 0, -134.58470153808594, 0, 0.01, 25.663541793823242]}]}


In [19]:
Map_LANDSAT8.addLayer(diff.randomVisualizer(),vis_params_diff, 'Squared Diff')
Map_LANDSAT8

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…